# Robustness Checking
This tutorial walks through how to use `DaCy`/`SpaCy` augmenters to evalutate robustness of any NLP pipeline. As an example we'll start out by evaluating SpaCy small and DaCy small on the test set of [DaNE](https://github.com/alexandrainst/danlp/blob/master/docs/docs/datasets.md#dane). DaNE is the Danish Dependency treebank tagged for part-of-speech tags, dependency relations and named entities. Lastly we will show how to use this framework on any other type of model using [DaNLP's BERT](https://github.com/alexandrainst/danlp/blob/master/docs/docs/tasks/ner.md#-bert-bert) as an example. 

Let us start of with installing the required packages and loading the models and dataset we wish to test on.


### Installing packages

In [1]:
#!pip install dacy
#!python -m spacy download da_core_news_sm

## Loading models and data

In [2]:
import spacy
import dacy

from dacy.datasets import dane

# load the DaNE test set
test = dane(splits=["test"])

# load models
spacy_small = spacy.load("da_core_news_sm")
dacy_small = dacy.load("small")

## Estimating performance
Evaluating models already in the `SpaCy` framework is very straightforward. Simply call the `score` function on your nlp pipeline and choose which metrics you want to calculate performance for. `score` is a wrapper for `SpaCy.scorer.Scorer` that outputs a nicely formatted dataframe. `score` calculates performance for NER, POS, tokenization, and dependency parsing by default, which can be changed with the score_fn argument.

In [3]:
from dacy.score import score

spacy_baseline = score(test, apply_fn=spacy_small, score_fn=["ents", "pos"])
dacy_baseline = score(test, apply_fn=dacy_small, score_fn=["ents", "pos"])

/Users/au561649/.virtualenvs/dacy/lib/python3.8/site-packages/spacy/pipeline/attributeruler.py:108: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True, as_spans=False)


In [4]:
spacy_baseline

,wall_time,ents_p,ents_r,ents_f,ents_per_type_LOC_p,ents_per_type_LOC_r,ents_per_type_LOC_f,ents_per_type_MISC_p,ents_per_type_MISC_r,ents_per_type_MISC_f,...,ents_per_type_PER_f,ents_per_type_ORG_p,ents_per_type_ORG_r,ents_per_type_ORG_f,ents_excl_MISC_ents_p,ents_excl_MISC_ents_r,ents_excl_MISC_ents_f,pos_acc,tag_acc,k
0,1.561627,0.715746,0.62724,0.668577,0.628319,0.739583,0.679426,0.660377,0.578512,0.61674,...,0.808743,0.72619,0.378882,0.497959,0.73107,0.640732,0.682927,0.948357,0.948357,0


In [5]:
dacy_baseline

,wall_time,ents_p,ents_r,ents_f,ents_per_type_LOC_p,ents_per_type_LOC_r,ents_per_type_LOC_f,ents_per_type_MISC_p,ents_per_type_MISC_r,ents_per_type_MISC_f,...,ents_per_type_PER_f,ents_per_type_ORG_p,ents_per_type_ORG_r,ents_per_type_ORG_f,ents_excl_MISC_ents_p,ents_excl_MISC_ents_r,ents_excl_MISC_ents_f,pos_acc,tag_acc,k
0,13.121968,0.774312,0.756272,0.765186,0.736364,0.84375,0.786408,0.656,0.677686,0.666667,...,0.90027,0.773109,0.571429,0.657143,0.809524,0.778032,0.793466,0.98002,0.0,0


### Estimating robustness and biases
To obtain performance estimates on augmented data, simply provide a list of augmenters as the `augmenters` argument. 

In [6]:
from dacy.augmenters import create_pers_augmenter
from dacy.datasets import female_names
from spacy.training.augment import create_lower_casing_augmenter

In [7]:
lower_aug = create_lower_casing_augmenter(level=1)
female_name_dict = female_names()
# Augmenter that replaces names with random Danish female names. Keep the format of the name as is (force_pattern_size=False)
# but replace the name with one of the two defined patterns
female_aug = create_pers_augmenter(
    female_name_dict,
    patterns=["fn,ln", "abbpunct,ln"],
    force_pattern_size=False,
    keep_name=False,
)

spacy_aug = score(
    test,
    apply_fn=spacy_small,
    score_fn=["ents", "pos"],
    augmenters=[lower_aug, female_aug],
)
dacy_aug = score(
    test,
    apply_fn=dacy_small,
    score_fn=["ents", "pos"],
    augmenters=[lower_aug, female_aug],
)

/Users/au561649/.virtualenvs/dacy/lib/python3.8/site-packages/spacy/pipeline/attributeruler.py:108: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True, as_spans=False)


In [8]:
import pandas as pd

pd.concat([spacy_baseline, spacy_aug])

,wall_time,ents_p,ents_r,ents_f,ents_per_type_LOC_p,ents_per_type_LOC_r,ents_per_type_LOC_f,ents_per_type_MISC_p,ents_per_type_MISC_r,ents_per_type_MISC_f,...,ents_per_type_PER_f,ents_per_type_ORG_p,ents_per_type_ORG_r,ents_per_type_ORG_f,ents_excl_MISC_ents_p,ents_excl_MISC_ents_r,ents_excl_MISC_ents_f,pos_acc,tag_acc,k
0,1.561627,0.715746,0.627240,0.668577,0.628319,0.739583,0.679426,0.660377,0.578512,0.616740,...,0.808743,0.726190,0.378882,0.497959,0.731070,0.640732,0.682927,0.948357,0.948357,0
0,1.643538,0.708738,0.261649,0.382199,0.666667,0.354167,0.462585,0.757143,0.438017,0.554974,...,0.362140,0.681818,0.093168,0.163934,0.683824,0.212815,0.324607,0.923838,0.923838,0
0,1.354588,0.669456,0.573477,0.617761,0.603448,0.729167,0.660377,0.676471,0.570248,0.618834,...,0.712166,0.592233,0.378882,0.462121,0.667553,0.574371,0.617466,0.946049,0.946448,0


In [9]:
pd.concat([dacy_baseline, dacy_aug])

,wall_time,ents_p,ents_r,ents_f,ents_per_type_LOC_p,ents_per_type_LOC_r,ents_per_type_LOC_f,ents_per_type_MISC_p,ents_per_type_MISC_r,ents_per_type_MISC_f,...,ents_per_type_PER_f,ents_per_type_ORG_p,ents_per_type_ORG_r,ents_per_type_ORG_f,ents_excl_MISC_ents_p,ents_excl_MISC_ents_r,ents_excl_MISC_ents_f,pos_acc,tag_acc,k
0,13.121968,0.774312,0.756272,0.765186,0.736364,0.84375,0.786408,0.656000,0.677686,0.666667,...,0.900270,0.773109,0.571429,0.657143,0.809524,0.778032,0.793466,0.980020,0.0,0
0,13.634849,0.727088,0.639785,0.680648,0.714286,0.78125,0.746269,0.614754,0.619835,0.617284,...,0.805797,0.686869,0.422360,0.523077,0.764228,0.645309,0.699752,0.974477,0.0,0
0,13.079645,0.749540,0.729391,0.739328,0.729730,0.84375,0.782609,0.672000,0.694215,0.682927,...,0.844444,0.708661,0.559006,0.625000,0.772727,0.739130,0.755556,0.979068,0.0,0


In the second row, we see that `SpaCy small` is very vulnerable to lower casing as NER recall drops from 0.66 to 0.38. `DaCy small` is slightly more robust lower casing, but still suffers. Changing names also leads to a drop in performance for both models. 

To better estimate the effect of stochastic augmenters such as those changing names or adding keystroke errors we can use the `k` argument in `score` to run the augmenter multiple times.

In [10]:
from dacy.augmenters import create_keyboard_augmenter

key_05_aug = create_keyboard_augmenter(
    doc_level=1, char_level=0.05, keyboard="QWERTY_DA"
)

spacy_key = score(
    test, apply_fn=spacy_small, score_fn=["ents", "pos"], augmenters=[key_05_aug], k=5
)

In [11]:
spacy_key

,wall_time,ents_p,ents_r,ents_f,ents_per_type_LOC_p,ents_per_type_LOC_r,ents_per_type_LOC_f,ents_per_type_MISC_p,ents_per_type_MISC_r,ents_per_type_MISC_f,...,ents_per_type_PER_f,ents_per_type_ORG_p,ents_per_type_ORG_r,ents_per_type_ORG_f,ents_excl_MISC_ents_p,ents_excl_MISC_ents_r,ents_excl_MISC_ents_f,pos_acc,tag_acc,k
0,1.816986,0.608871,0.541219,0.573055,0.512195,0.656250,0.575342,0.589474,0.462810,0.518519,...,0.697548,0.604396,0.341615,0.436508,0.613466,0.562929,0.587112,0.844557,0.844557,0
1,1.531972,0.588469,0.530466,0.557964,0.558559,0.645833,0.599034,0.552381,0.479339,0.513274,...,0.661376,0.573034,0.316770,0.408000,0.597990,0.544622,0.570060,0.850424,0.850424,1
2,1.520552,0.612288,0.517921,0.561165,0.558559,0.645833,0.599034,0.556818,0.404959,0.468900,...,0.681319,0.606742,0.335404,0.432000,0.625000,0.549199,0.584653,0.850514,0.850514,2
3,1.634042,0.612121,0.543011,0.575499,0.580952,0.635417,0.606965,0.469027,0.438017,0.452991,...,0.718232,0.621053,0.366460,0.460938,0.654450,0.572082,0.610501,0.847199,0.847199,3
4,1.465947,0.588353,0.525090,0.554924,0.549180,0.697917,0.614679,0.520000,0.429752,0.470588,...,0.688000,0.555556,0.279503,0.371901,0.605528,0.551487,0.577246,0.846414,0.846414,4


In this manner, evaluating performance on augmented data for SpaCy pipelines is as easy as defining the augmenters and calling a single function. In the `dacy_paper_replication.py` script you can find the exact script used to evaluate the robustness of Danish NLP models in the [DaCy paper]().

# Evaluating custom models
Evaluating models not in the `SpaCy` framework requires the user to write an `apply_fn` that takes a series of SpaCy `Example`s as input, and applies their model to it and returns list of examples `Example`. 

The following shows how to write one for the NERDA model for named entity recognition. Notice that we replace the tokenizer with the spaCy tokenizer (where they use the NLTK) it turns out that this provides a better performance.

We will start out by installing the package and downloading the model. Then we will define an apply function which converts the models tags to spacy annotations.

In [12]:
# !pip install NERDA

In [17]:
from NERDA.precooked import DA_BERT_ML
import ssl

model = DA_BERT_ML()
# to download the danlp and nerda you will have to set up a certificate:
ssl._create_default_https_context = ssl._create_unverified_context
# model.download_network()
model.load_network()

Device automatically set to: cpu


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



        Model loaded. Please make sure, that you're running the latest version 
        of 'NERDA' otherwise the model is not guaranteed to work.
        


In [20]:
from typing import Iterable, List
from spacy.tokens import Doc, Span
from spacy.training import Example

# set up a danish tokenization pipeline
nlp_da = spacy.blank("da")


def add_iob(doc: Doc, iob: List[str]) -> Doc:
    """A helper function for adding iob tags to Doc

    Args:
        doc (Doc): A SpaCy doc
        iob (List[str]): a list of tokens on the IOB format

    Returns:
        Doc: A doc with the spans to the new IOB
    """
    ent = []
    for i, label in enumerate(iob):

        # turn OOB labels into spans
        if label == "O":
            continue
        iob_, ent_type = label.split("-")
        if (i - 1 >= 0 and iob_ == "I" and iob[i - 1] == "O") or (
            i == 0 and iob_ == "I"
        ):
            iob_ = "B"
        if iob_ == "B":
            start = i
        if i + 1 >= len(iob) or iob[i + 1].split("-")[0] != "I":
            ent.append(Span(doc, start, i + 1, label=ent_type))
    doc.set_ents(ent)
    return doc


def apply_nerda(examples: Iterable[Example]) -> List[Example]:
    sentences = []
    docs_y = []
    for example in examples:
        # tokenization
        # they use NLTK for their tokenization,
        # but turns out that the spacy tokenizer provides better results
        sentences.append([t.text for t in nlp_da(example.reference.text)])
        docs_y.append(example.reference)

    # ner
    labels = model.predict(sentences=sentences)

    examples_ = []
    for doc_y, label, words in zip(docs_y, labels, sentences):
        if len(label) < len(words):
            label += ["O"] * (len(words) - len(label))

        doc = Doc(nlp_da.vocab, words=words)
        doc = add_iob(doc, iob=label)
        examples_.append(Example(doc, doc_y))
    return examples_

In [21]:
nerda = score(test, apply_fn=apply_nerda, score_fn=["ents"])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
nerda

,wall_time,ents_p,ents_r,ents_f,ents_per_type_LOC_p,ents_per_type_LOC_r,ents_per_type_LOC_f,ents_per_type_MISC_p,ents_per_type_MISC_r,ents_per_type_MISC_f,ents_per_type_PER_p,ents_per_type_PER_r,ents_per_type_PER_f,ents_per_type_ORG_p,ents_per_type_ORG_r,ents_per_type_ORG_f,ents_excl_MISC_ents_p,ents_excl_MISC_ents_r,ents_excl_MISC_ents_f,k
0,195.918393,0.819231,0.763441,0.790353,0.747826,0.895833,0.815166,0.756757,0.694215,0.724138,0.942197,0.905556,0.923513,0.768595,0.57764,0.659574,0.836186,0.782609,0.808511,0


If you are in doubt how to create an apply function for your model you can find more inspiration in [`papers/DaCy../apply_fns`](https://github.com/centre-for-humanities-computing/DaCy/tree/main/papers/DaCy-A-Unified-Framework-for-Danish-NLP/apply_fns). This folder contains apply functions for DaNLP's BERT, Flair, NERDA, and Polyglot. Otherwise, feel free to open an issue on the GitHub. 